**Map over Copenhagen**

ImportError: cannot import name 'CRS' from 'pyproj' (/opt/anaconda3/lib/python3.7/site-packages/pyproj/__init__.py)

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as pd
import matplotlib.yplot as plt
mpltoolkits.basemap

ImportError: cannot import name 'CRS' from 'pyproj' (/opt/anaconda3/lib/python3.7/site-packages/pyproj/__init__.py)

In [3]:
#Henter geo-info om de danske Postnumre
#url = "https://raw.githubusercontent.com/ok-dk/dagi/master/geojson/postnumre.geojson"
#postnr = gpd.read_file(url)
#Set to meter based system
#postnr.to_crs(epsg = '25832', inplace = True)
#postnr.columns = postnr.columns.str.lower()
#postnr.rename(columns = {'postnr_txt':'postnr'}, inplace = True)

#geoplot.polyplot(postnr, figsize=(8, 4))
#print(postnr)


#Average score rating per postal code


NameError: name 'gpd' is not defined

In [6]:
#Henter geo-info om de danske Postnumre
url = "https://raw.githubusercontent.com/ok-dk/dagi/master/geojson/postnumre.geojson"
postnr = gpd.read_file(url)
#Set to meter based system
postnr.to_crs(epsg = '25832', inplace = True)
postnr.columns = postnr.columns.str.lower()
postnr.rename(columns = {'postnr_txt':'postnr'}, inplace = True)
 
# Information for Postnr
postnr_temp = pd.read_json('https://dawa.aws.dk/postnumre')
postnr_loop = []
# kommune-info er i en kolonnen, hvor hver obs er en liste af dicts - vil have kominfo ud
for idx, row in postnr_temp.iterrows():
    for dct in row['kommuner']:
        dct['postnr'] = row['nr']
        dct['post_navn'] = row['navn']
        postnr_loop.append(dct)
 
post_info = pd.DataFrame(postnr_loop).loc[:,['postnr','post_navn','kode']].rename(columns = {'kode':'komkode'})
post_info['postnr']=post_info['postnr'].astype(str)
 
# information for Kommuner
kommune_info = pd.read_json('https://dawa.aws.dk/kommuner')\
                .pipe(lambda df: \
                        df.assign(komkode=df.kode.astype(str).str.zfill(4)))\
                .loc[:,['komkode', 'navn', 'regionskode']]\
                .rename(columns = {'navn':'kommune'})
 
kommune_info['regionskode'] = kommune_info['regionskode'].astype(str)
               
 
# Information for Regioner
region_info = pd.read_json('https://dawa.aws.dk/regioner/')\
                .loc[:,['kode','navn']]\
                .add_prefix('regions')
region_info['regionskode'] = region_info['regionskode'].astype(str)
region_info.rename(columns = {'regionsnavn':'region'}, inplace = True)
 
#Gather in df with geo-coordinates:
postnr = postnr.merge(post_info, how='left')
 
#Manuel erstatning af kommunekoder - I kbh var nogle af postnumrene slået sammen
# således af kbhk var 1000-1499, hvilket DAWA data derfor ikke kunne merge kommunekoder på.
postnr['komkode'] = np.where(postnr.postbynavn == 'København K','0101',postnr.komkode)
postnr['komkode'] = np.where(postnr.postbynavn == 'København V','0101',postnr.komkode)
postnr['komkode'] = np.where(postnr.postbynavn == 'Frederiksberg C','0147',postnr.komkode)
 
#Merger kommune og regionsdata
postnr = postnr.merge(kommune_info,how='left')\
                .merge(region_info,how='left')
 
postnr.head()


data2 = postnr.copy()
data2.set_index('postnr')
data2 = data2.sort_values('postnr')
data2 = data2.reset_index(drop = True)
#data2.reset_index(inplace=True)

postnr2 = data2.iloc[0:16]



NameError: name 'gpd' is not defined